# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/qTEA1/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298426091608                   -0.85    5.1         
  2   -8.300204100077       -2.75       -1.25    1.0    124ms
  3   -8.300433901350       -3.64       -1.89    2.0    140ms
  4   -8.300460433221       -4.58       -2.75    3.1    217ms
  5   -8.300463561428       -5.50       -3.02    8.6    268ms
  6   -8.300464082102       -6.28       -3.21    2.2    156ms
  7   -8.300464364030       -6.55       -3.35    1.9    167ms
  8   -8.300464510312       -6.83       -3.50    2.4    170ms
  9   -8.300464610737       -7.00       -3.72    2.8    228ms
 10   -8.300464627343       -7.78       -3.85    1.4    210ms
 11   -8.300464643077       -7.80       -4.19    2.9    206ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67452691947                   -0.70    6.6         
  2   -16.67859930430       -2.39       -1.14    1.0    320ms
  3   -16.67920523572       -3.22       -1.87    3.8    407ms
  4   -16.67928012437       -4.13       -2.76    3.5    388ms
  5   -16.67928600366       -5.23       -3.19    5.0    519ms
  6   -16.67928620258       -6.70       -3.49    2.2    358ms
  7   -16.67928621788       -7.82       -3.96    1.6    297ms
  8   -16.67928622142       -8.45       -4.58    2.9    377ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.30766304143                   -0.56    7.6         
  2   -33.33465351474       -1.57       -1.00    1.5    1.24s
  3   -33.33601483381       -2.87       -1.73    5.2    1.62s
  4   -33.33616359672       -3.83       -2.63    5.4    1.62s
  5   -33.33689090235       -3.14       -2.47    6.5    1.79s
  6   -33.33692375387       -4.48       -2.69    2.8    1.21s
  7   -33.33694363936       -4.70       -3.62    2.5    1.34s
  8   -33.33694370859       -7.16       -3.72    4.0    1.47s
  9   -33.33694377199       -7.20       -4.17    1.4    1.00s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298441468215                   -0.85    5.1         
  2   -8.300246647942       -2.74       -1.59    1.0   86.0ms
  3   -8.300442104331       -3.71       -2.68    3.1    136ms
  4   -8.300443759320       -5.78       -2.60    4.2    168ms
  5   -8.300463937843       -4.70       -3.29    1.0   92.7ms
  6   -8.300464569524       -6.20       -3.75    2.9    172ms
  7   -8.300464641336       -7.14       -4.52    1.6    119ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32588571488                   -0.56    7.1         
  2   -33.31775454153   +   -2.09       -1.27    1.0    947ms
  3   -17.12485769469   +    1.21       -0.46    5.9    1.91s
  4   -33.31309371157        1.21       -1.73    6.6    1.98s
  5   -32.85448142944   +   -0.34       -1.15    3.4    1.56s
  6   -31.83900390896   +    0.01       -0.97    5.1    1.70s
  7   -33.33274861636        0.17       -2.06    4.0    1.45s
  8   -33.32168238470   +   -1.96       -1.81    2.9    1.33s
  9   -33.33673632562       -1.82       -2.58    2.8    1.33s
 10   -33.33674377678       -5.13       -2.82    2.9    1.28s
 11   -33.33692969084       -3.73       -3.29    3.0    1.24s
 12   -33.33694030495       -4.97      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.